# PCA on MovieLens

### Download ml-100k dataset

In [3]:
%%sh
wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
unzip -o ml-100k.zip

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


--2021-06-19 13:11:24--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.1’

     0K .......... .......... .......... .......... ..........  1%  545K 9s
    50K .......... .......... .......... .......... ..........  2% 1018K 7s
   100K .......... .......... .......... .......... ..........  3% 80.1M 4s
   150K .......... .......... .......... .......... ..........  4%  172M 3s
   200K .......... .......... .......... .......... ..........  5% 1.00M 3s
   250K .......... .......... .......... .......... ..........  6%  140M 3s
   300K .......... .......... .......... .......... ..........  7% 61.8M 2s
   350K .......... .......... .......... .......... ..........  8% 1.02M 3s
   400K .......... .......... ....

In [4]:
%cd ml-100k
!shuf ua.base -o ua.base.shuffled
!head -5 ua.base.shuffled

/root/Learn-Amazon-SageMaker/sdkv2/ch4/ml-100k
420	475	4	891357162
393	690	4	887742110
221	70	3	875245870
334	154	4	891547235
907	596	4	880159015


### Build training set and test set

In [5]:
num_users=943
num_movies=1682
num_features=num_users+num_movies

num_ratings_train=90570
num_ratings_test=9430

In [6]:
import csv
import numpy as np
from scipy.sparse import lil_matrix

def loadDataset(filename, lines, columns):
    # Features are one-hot encoded in a sparse matrix
    X = lil_matrix((lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line=0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter='\t')
        for userId,movieId,rating,timestamp in samples:
            X[line,int(userId)-1] = 1
            X[line,int(num_users)+int(movieId)-1] = 1
            Y.append(int(rating))
            line=line+1       
    Y=np.array(Y).astype('float32')
    return X,Y

In [7]:
X_train, Y_train = loadDataset('ua.base.shuffled', num_ratings_train, num_features)
X_test, Y_test = loadDataset('ua.test', num_ratings_test, num_features)

In [8]:
print(X_train.shape)
print(Y_train.shape)
assert X_train.shape == (num_ratings_train, num_features)
assert Y_train.shape == (num_ratings_train, )

print(X_test.shape)
print(Y_test.shape)
assert X_test.shape  == (num_ratings_test, num_features)
assert Y_test.shape  == (num_ratings_test, )

(90570, 2625)
(90570,)
(9430, 2625)
(9430,)


### Convert to protobuf and save to S3

In [9]:
import sagemaker

bucket = sagemaker.Session().default_bucket()
prefix = 'adeelml-fm-movielens'

train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

In [10]:
import io, boto3
import sagemaker.amazon.common as smac

def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, X, Y)
    # use smac.write_numpy_to_dense_tensor(buf, feature, label) for numpy arrays
    buf.seek(0)
    print(buf)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    
train_data = writeDatasetToProtobuf(X_train, Y_train, bucket, train_prefix, train_key)    
test_data  = writeDatasetToProtobuf(X_test, Y_test, bucket, test_prefix, test_key)    
  
print(train_data)
print(test_data)
print('Output: {}'.format(output_prefix))

s3://sagemaker-us-west-1-886035371869/adeelml-fm-movielens/train/train.protobuf
s3://sagemaker-us-west-1-886035371869/adeelml-fm-movielens/test/test.protobuf
Output: s3://sagemaker-us-west-1-886035371869/adeelml-fm-movielens/output


### Run training job

In [11]:
import boto3
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('pca', region)

In [ ]:
pca = sagemaker.estimator.Estimator(container,
                                   role=sagemaker.get_execution_role(),
                                   instance_count=1, 
                                   instance_type='ml.c5.xlarge',
                                   output_path=output_prefix
                                   )

pca.set_hyperparameters(feature_dim=num_features,
                      num_components=64,
                      mini_batch_size=1024)

pca.fit({'train': train_data, 'test': test_data})

2021-06-19 13:11:47 Starting - Starting the training job...
2021-06-19 13:11:49 Starting - Launching requested ML instancesProfilerReport-1624108307: InProgress
......
2021-06-19 13:13:13 Starting - Preparing the instances for training.........

### Deploy model

In [ ]:
endpoint_name = 'pca-movielens-100k'
pca_predictor = pca.deploy(endpoint_name=endpoint_name,
                         instance_type='ml.t2.medium', initial_instance_count=1)

In [ ]:
import json

from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer

class PCASerializer(JSONSerializer):
    def serialize(self, data):
       js = {'instances': []}
       for row in data:
              js['instances'].append({'features': row.tolist()})
       return json.dumps(js)

pca_predictor.serializer = PCASerializer()
pca_predictor.deserializer = JSONDeserializer()

### Run predictions

In [ ]:
result = pca_predictor.predict(X_test[0].toarray())
print(result)

In [ ]:
pca_predictor.delete_endpoint()